In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

SEED=9

In [3]:
df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

In [4]:
df.shape

(8693, 14)

In [5]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [6]:
# to drop: PassengerId,Name, Age
df.drop(['PassengerId','Name'],axis=1,inplace=True)

In [7]:
df[['Cabin_deck', 'Cabin_num','Cabin_side']] = df['Cabin'].str.split('/', expand=True)
df.drop('Cabin',axis=1,inplace=True)

In [8]:
df=df[['HomePlanet', 'CryoSleep', 'Cabin_deck','Cabin_num','Cabin_side', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','Transported']]

In [9]:
df.head()

,HomePlanet,CryoSleep,Cabin_deck,Cabin_num,Cabin_side,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B,0,P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F,0,S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A,0,S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A,0,S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F,1,S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [10]:
df.isna().sum()

HomePlanet      201
CryoSleep       217
Cabin_deck      199
Cabin_num       199
Cabin_side      199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64

## Missing values 

In [11]:
#Done
#CryoSleep
columns_to_check = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

for index, row in df.iterrows():
    if pd.isnull(row['CryoSleep']):
        if df[columns_to_check].sum(axis=1)[index] == 0:
            df.loc[index, 'CryoSleep'] = True
        else:
            df.loc[index, 'CryoSleep'] = False
            

In [12]:
#imputing randomly xD
df['Cabin_num']=df['Cabin_num'].fillna(np.random.randint(0,1890))

In [13]:
#Its seems VIP pssengers tend to spend more 
df.groupby('VIP')['FoodCourt'].mean()

VIP
False     426.336536
True     1811.393782
Name: FoodCourt, dtype: float64

In [14]:
df['FoodCourt'].mean()

458.07720329024676

In [15]:
#VIP
food_court_mean = df['FoodCourt'].mean()
for index, row in df.iterrows():
    if pd.isnull(row['VIP']):
        if df['HomePlanet'][index] == 'Earth':
            df.loc[index, 'VIP'] = False
        elif df['FoodCourt'][index] > food_court_mean :
            df.loc[index, 'VIP'] = True
        else:
            df.loc[index, 'VIP'] = False

In [16]:
df.groupby('HomePlanet')['Cabin_deck'].value_counts()

HomePlanet  Cabin_deck
Earth       G             2498
            F             1614
            E              395
Europa      B              766
            C              734
            A              252
            D              186
            E              128
            T                4
Mars        F             1110
            E              330
            D              282
Name: Cabin_deck, dtype: int64

In [17]:
#Cabin_deck
for index, row in df.iterrows():
    if pd.isnull(row['Cabin_deck']):
        if df['HomePlanet'][index] == 'Earth':
            df.loc[index, 'Cabin_deck'] = 'G'
        elif df['HomePlanet'][index] == 'Europa':
            df.loc[index, 'Cabin_deck'] = 'B'
        else:
            df.loc[index, 'Cabin_deck'] = 'F'

In [18]:
#Cabin_side seems to be distributed randomly 
#Cabin_side
'''for index, row in df.iterrows():
    if pd.isnull(row['Cabin_side']):
        if np.random.rand() > 0.5:
            df.loc[index, 'Cabin_side'] = 'S'
        else:
            df.loc[index, 'Cabin_side'] = 'P'''

"for index, row in df.iterrows():\n    if pd.isnull(row['Cabin_side']):\n        if np.random.rand() > 0.5:\n            df.loc[index, 'Cabin_side'] = 'S'\n        else:\n            df.loc[index, 'Cabin_side'] = 'P"

In [19]:
df.drop('Cabin_side',axis=1,inplace=True)

In [20]:
# impute the missing values for passengers in cryosleep  

df['RoomService'] = df.apply(lambda row: 0 if row['CryoSleep'] == 1 else row['RoomService'], axis=1)
df['FoodCourt'] = df.apply(lambda row: 0 if row['CryoSleep'] == 1 else row['FoodCourt'], axis=1)
df['ShoppingMall'] = df.apply(lambda row: 0 if row['CryoSleep'] == 1 else row['ShoppingMall'], axis=1)
df['Spa'] = df.apply(lambda row: 0 if row['CryoSleep'] == 1 else row['Spa'], axis=1)
df['VRDeck'] = df.apply(lambda row: 0 if row['CryoSleep'] == 1 else row['VRDeck'], axis=1)

In [21]:
#most_common_value_dest = df['Destination'].mode()[0]

# Fill missing values with the most common value.
#df['Destination'].fillna(most_common_value_dest, inplace=True)

In [22]:
df.groupby('VIP')['Age'].mean()

VIP
False    28.601519
True     37.363636
Name: Age, dtype: float64

In [23]:
#imputing age with mean depending if they are VIP or not since it seems that VIPs are older than non-VIP
category_means_vip = df.groupby('VIP').mean()

for index, row in df.iterrows():
    category = row['VIP']
    if pd.isnull(row['Age']):
        df.loc[index, 'Age'] = category_means_vip.loc[category, 'Age']

/tmp/ipykernel_32/1614470757.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  category_means_vip = df.groupby('VIP').mean()


In [24]:
#impute missing values for passengers that are not in CryoSleep with mean depanding if they are VIP or not since VIP sepnd more on average

for index, row in df.iterrows():
    category = row['VIP']
    if pd.isnull(row['RoomService']):
        df.loc[index, 'RoomService'] = category_means_vip.loc[category, 'RoomService']
    if pd.isnull(row['FoodCourt']):
        df.loc[index, 'FoodCourt'] = category_means_vip.loc[category, 'FoodCourt']
    if pd.isnull(row['ShoppingMall']):
        df.loc[index, 'ShoppingMall'] = category_means_vip.loc[category, 'ShoppingMall']
    if pd.isnull(row['Spa']):
        df.loc[index, 'Spa'] = category_means_vip.loc[category, 'Spa']
    if pd.isnull(row['VRDeck']):
        df.loc[index, 'VRDeck'] = category_means_vip.loc[category, 'VRDeck']

In [25]:
# didnt find any clear identification on how ot know the HomePlanet
for index, row in df.iterrows():
    if pd.isnull(row['HomePlanet']):
        if df['Destination'][index] == '55 Cancri e':
            df.loc[index, 'HomePlanet'] = 'Europa'
        else:
            df.loc[index, 'HomePlanet'] = 'Earth'

In [26]:
#TODO need to redo it need deeper analysis
for index, row in df.iterrows():
    if pd.isnull(row['Destination']):
        if df['HomePlanet'][index] == 'Earth':
            df.loc[index, 'Destination'] = 'TRAPPIST-1e'
        else:
            df.loc[index, 'Destination'] = '55 Cancri e'

In [27]:
df.groupby('VIP')['Cabin_deck'].value_counts()

VIP    Cabin_deck
False  F             2804
       G             2654
       E              859
       B              791
       C              696
       D              444
       A              219
       T                5
True   C               51
       B               49
       A               37
       D               34
       F               33
       E               17
Name: Cabin_deck, dtype: int64

In [28]:
df.isna().sum()

HomePlanet      0
CryoSleep       0
Cabin_deck      0
Cabin_num       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [29]:
df['CryoSleep'] = df['CryoSleep'].apply(lambda x: 1 if x==True else 0)
df['Transported'] = df['Transported'].apply(lambda x: 1 if x==True else 0)
df['VIP'] = df['VIP'].apply(lambda x: 1 if x==True else 0)
#df['Cabin_side'] = df['Cabin_side'].apply(lambda x: 1 if x=='S' else 0)

df['Cabin_num'] = df['Cabin_num'].astype(int)

In [30]:
df['CryoSleep'].value_counts()

0    5558
1    3135
Name: CryoSleep, dtype: int64

In [31]:
df = pd.get_dummies(df,drop_first=True)

In [32]:
y=df['Transported']
X=df.drop('Transported',axis=1)

In [33]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.2,random_state=SEED)

# XGBoost

In [34]:
params_1 = {'subsample': 0.8, 'reg_lambda': 0.9, 'reg_alpha': 0.3, 'n_estimators': 300, 'min_child_weight': 4, 'max_depth': 5, 'learning_rate': 0.05, 'gamma': 0.5, 'colsample_bytree': 0.6}

In [102]:
import xgboost as xgb

# tree_method='gpu_hist'
xgb_classifier = xgb.XGBClassifier(tree_method='gpu_hist',objective="binary:logistic",random_state=SEED,**params_1)

In [100]:
xgb_classifier.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=9, ...)

In [37]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': np.arange(100, 1000, 100),  # Values from 500 to 900 in steps of 50
    'learning_rate': np.linspace(0.01, 0.1, num=10),  # Values from 0.01 to 0.1, 10 evenly spaced values
    'max_depth': np.arange(1, 11),  # Values from 1 to 10
    'min_child_weight': np.arange(1, 6),  # Values from 1 to 5
    'subsample': np.arange(0.5, 1.0, 0.1),  # Values from 0.5 to 0.9 in steps of 0.1
    'colsample_bytree': np.arange(0.5, 1.0, 0.1),  # Values from 0.5 to 0.9 in steps of 0.1
    'gamma': np.linspace(0, 1, num=11),  # Values from 0 to 1, 11 evenly spaced values
    'reg_alpha': np.linspace(0, 1, num=11),  # Values from 0 to 1, 11 evenly spaced values
    'reg_lambda': np.linspace(0, 1, num=11)  # Values from 0 to 1, 11 evenly spaced values
}

In [38]:
xgb_random_search = RandomizedSearchCV(xgb_classifier, param_distributions=param_dist, n_iter=50, scoring='accuracy', n_jobs=-1, cv=5, verbose=3, random_state=SEED)

In [39]:
xgb_random_search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END colsample_bytree=0.6, gamma=0.5, learning_rate=0.04000000000000001, max_depth=7, min_child_weight=4, n_estimators=100, reg_alpha=0.4, reg_lambda=0.30000000000000004, subsample=0.5;, score=0.815 total time=   1.5s
[CV 3/5] END colsample_bytree=0.6, gamma=0.5, learning_rate=0.04000000000000001, max_depth=7, min_child_weight=4, n_estimators=100, reg_alpha=0.4, reg_lambda=0.30000000000000004, subsample=0.5;, score=0.794 total time=   0.7s
[CV 5/5] END colsample_bytree=0.6, gamma=0.5, learning_rate=0.04000000000000001, max_depth=7, min_child_weight=4, n_estimators=100, reg_alpha=0.4, reg_lambda=0.30000000000000004, subsample=0.5;, score=0.804 total time=   0.7s
[CV 2/5] END colsample_bytree=0.5, gamma=0.2, learning_rate=0.1, max_depth=6, min_child_weight=4, n_estimators=600, reg_alpha=1.0, reg_lambda=0.8, subsample=0.6;, score=0.816 total time=   3.5s
[CV 4/5] END colsample_bytree=0.5, gamma=0.2, learning_rate=0.1, m

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                        'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'min_child_weight': array([1, 2, 3, 4, 5]),
                                        'n_estimators': array([100, 200, 300, 400, 500, 600, 700, 800, 900]),
                                        'reg_alpha': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'reg_lambda': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])},
                   random_state=9, scoring='accuracy', verbose=3)

In [40]:
print(xgb_random_search.best_params_)
print(xgb_random_search.best_score_)

{'subsample': 0.8999999999999999, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.5, 'n_estimators': 600, 'min_child_weight': 5, 'max_depth': 5, 'learning_rate': 0.06000000000000001, 'gamma': 0.1, 'colsample_bytree': 0.7}
0.8101795199354536


In [41]:
from sklearn.metrics import accuracy_score

In [42]:
best_model = xgb_random_search.best_estimator_
xgb_preds = best_model.predict(X_test)
print(accuracy_score(y_test, xgb_preds))

0.7763082231167338


In [43]:
#xgb_preds = xgb_classifier.predict(X_test)
#print(accuracy_score(xgb_preds,y_test))

# TF-DF

In [32]:
!pip install tensorflow tensorflow_decision_forests

In [33]:
np.random.seed=SEED
def split_dataset(dataset, test_ratio=0.20):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, test_ds_pd = split_dataset(df)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

7043 examples in training, 1650 examples in testing.


In [41]:
import tensorflow_decision_forests as tfdf

train_data = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label='Transported')
test_data = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label='Transported')
data=tfdf.keras.pd_dataframe_to_tf_dataset(df, label='Transported')

In [42]:
tf_df = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.CLASSIFICATION)

tf_df.compile(['accuracy'])
tf_df.fit(train_data)

Use /tmp/tmpufn5e3ob as temporary training directory
Reading training dataset...


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/tensorflow_decision_forests/keras/core.py", line 1286, in _consumes_training_examples_until_eof  *
        num_examples += self.train_step(data)
    File "/opt/conda/lib/python3.10/site-packages/tensorflow_decision_forests/keras/core.py", line 615, in _method_wrapper  *
        result = method(self, *args, **kwargs)
    File "/opt/conda/lib/python3.10/site-packages/tensorflow_decision_forests/keras/core.py", line 616, in train_step  *
        return self.collect_data_step(data, is_training_example=True)
    File "/opt/conda/lib/python3.10/site-packages/tensorflow_decision_forests/keras/core.py", line 615, in _method_wrapper  *
        result = method(self, *args, **kwargs)
    File "/opt/conda/lib/python3.10/site-packages/tensorflow_decision_forests/keras/core.py", line 754, in collect_data_step  *
        normalized_semantic_inputs = tf_core.normalize_inputs(
    File "/opt/conda/lib/python3.10/site-packages/

In [ ]:
predictions = tf_df.predict(test_data)
y_true      = test_ds_pd["Transported"]

from sklearn.metrics import accuracy_score
print(accuracy_score(y_true,predictions))

In [ ]:
tf_df.evaluate(test_data)

# Submission

In [41]:
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [42]:
test.shape

(4277, 13)

In [43]:
test_id = test.PassengerId
test.drop(['PassengerId','Name'],axis=1,inplace=True)

In [44]:
test[['Cabin_deck', 'Cabin_num','Cabin_side']] = test['Cabin'].str.split('/', expand=True)
test.drop('Cabin',axis=1,inplace=True)

In [45]:
test=test[['HomePlanet', 'CryoSleep', 'Cabin_deck','Cabin_num','Cabin_side', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]

In [46]:
test['Cabin_num']=test['Cabin_num'].fillna(np.random.randint(0,1890))

In [47]:
#VIP
for index, row in test.iterrows():
    if pd.isnull(row['VIP']):
        if test['HomePlanet'][index] == 'Earth':
            test.loc[index, 'VIP'] = False
        elif test['FoodCourt'][index] > food_court_mean:
            test.loc[index, 'VIP'] = True
        else:
            test.loc[index, 'VIP'] = False

In [48]:
#Cabin_deck
for index, row in test.iterrows():
    if pd.isnull(row['Cabin_deck']):
        if test['HomePlanet'][index] == 'Earth':
            test.loc[index, 'Cabin_deck'] = 'G'
        elif test['HomePlanet'][index] == 'Europa':
            test.loc[index, 'Cabin_deck'] = 'B'
        else:
            test.loc[index, 'Cabin_deck'] = 'F'

In [49]:
#Cabin_side seems to be distributed randomly 
#Cabin_side
'''for index, row in test.iterrows():
    if pd.isnull(row['Cabin_side']):
        if np.random.rand() > 0.5:
            test.loc[index, 'Cabin_side'] = 'S'
        else:
            test.loc[index, 'Cabin_side'] = 'P'''

"for index, row in test.iterrows():\n    if pd.isnull(row['Cabin_side']):\n        if np.random.rand() > 0.5:\n            test.loc[index, 'Cabin_side'] = 'S'\n        else:\n            test.loc[index, 'Cabin_side'] = 'P"

In [50]:
test.drop('Cabin_side',axis=1,inplace=True)

In [51]:
columns_to_check = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

for index, row in test.iterrows():
    if pd.isnull(row['CryoSleep']):
        if test[columns_to_check].sum(axis=1)[index] == 0:
            test.loc[index, 'CryoSleep'] = True
        else:
            test.loc[index, 'CryoSleep'] = False

In [52]:
# impute the missing values for passengers in cryosleep  

test['RoomService'] = test.apply(lambda row: 0 if row['CryoSleep'] == 1 else row['RoomService'], axis=1)
test['FoodCourt'] = test.apply(lambda row: 0 if row['CryoSleep'] == 1 else row['FoodCourt'], axis=1)
test['ShoppingMall'] = test.apply(lambda row: 0 if row['CryoSleep'] == 1 else row['ShoppingMall'], axis=1)
test['Spa'] = test.apply(lambda row: 0 if row['CryoSleep'] == 1 else row['Spa'], axis=1)
test['VRDeck'] = test.apply(lambda row: 0 if row['CryoSleep'] == 1 else row['VRDeck'], axis=1)

In [53]:
for index, row in test.iterrows():
    category = row['VIP']
    if pd.isnull(row['Age']):
        test.loc[index, 'Age'] = category_means_vip.loc[category, 'Age']

In [54]:
#impute missing values for passengers that are not in CryoSleep with mean depanding if they are VIP or not since VIP sepnd more on average

for index, row in df.iterrows():
    category = row['VIP']
    if pd.isnull(row['RoomService']):
        test.loc[index, 'RoomService'] = category_means_vip.loc[category, 'RoomService']
    if pd.isnull(row['FoodCourt']):
        test.loc[index, 'FoodCourt'] = category_means_vip.loc[category, 'FoodCourt']
    if pd.isnull(row['ShoppingMall']):
        test.loc[index, 'ShoppingMall'] = category_means_vip.loc[category, 'ShoppingMall']
    if pd.isnull(row['Spa']):
        test.loc[index, 'Spa'] = category_means_vip.loc[category, 'Spa']
    if pd.isnull(row['VRDeck']):
        test.loc[index, 'VRDeck'] = category_means_vip.loc[category, 'VRDeck']
        

In [55]:
# didnt find any clear identification on how ot know the HomePlanet
for index, row in test.iterrows():
    if pd.isnull(row['HomePlanet']):
        if test['Destination'][index] == '55 Cancri e':
            test.loc[index, 'HomePlanet'] = 'Europa'
        else:
            test.loc[index, 'HomePlanet'] = 'Earth'

In [56]:
#TODO need to redo it need deeper analysis
for index, row in test.iterrows():
    if pd.isnull(row['Destination']):
        if test['HomePlanet'][index] == 'Earth':
            test.loc[index, 'Destination'] = 'TRAPPIST-1e'
        else:
            test.loc[index, 'Destination'] = '55 Cancri e'

In [57]:
test['CryoSleep'] = test['CryoSleep'].apply(lambda x: 1 if x==True else 0)
test['VIP'] = test['VIP'].apply(lambda x: 1 if x==True else 0)
#test['Cabin_side'] = test['Cabin_side'].apply(lambda x: 1 if x=='S' else 0)

test['Cabin_num'] = test['Cabin_num'].astype(int)

In [58]:
test = pd.get_dummies(test,drop_first=True)

In [69]:
test.shape

(4277, 20)

In [103]:
#xgb_classifier.fit(X,y)add_prefix

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=9, ...)

In [104]:
xgb_preds = xgb_classifier.predict(test)

In [105]:
my_submissions = pd.DataFrame({'PassengerId':test_id,'Transported':xgb_preds})

In [106]:
my_submissions['Transported'] = my_submissions['Transported'].apply(lambda x: True if x==1 else False)

In [107]:
my_submissions.to_csv('/kaggle/working/submission.csv', index=False)
my_submissions.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
